In [16]:
from __future__ import print_function

In [17]:
# We'll need numpy for some mathematical operations
import numpy as np

# matplotlib for displaying the output
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib inline

In [18]:
# and IPython.display for audio output
from IPython.display import Audio

# Librosa for audio
import librosa
# And the display module for visualization
import librosa.display

In [19]:
audio_path = librosa.util.example_audio_file()
y, sr = librosa.load("C:/Users/Ujjval/Downloads/ESC-50-master/ESC-50-master/audio/1-7057-A-12.wav")

In [20]:
Audio(data = y, rate = sr)

In [65]:
# Let's make and display a mel-scaled power (energy-squared) spectrogram
S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
# Convert to log scale (dB). We'll use the peak power (max) as reference.
log_S = librosa.power_to_db(S, ref=np.max)
# Audio pitch
print (log_S)

[[-18.57305482 -18.68916954 -22.50212796 ..., -21.61341251 -24.29107039
  -26.57180892]
 [-20.32817837 -16.79926285 -18.92391476 ..., -20.35279943 -17.34771407
  -19.94018663]
 [-20.56881959 -15.63588535 -10.2440247  ..., -13.63524223 -13.17078714
  -14.65898603]
 ..., 
 [-54.82829057 -49.95413087 -46.8234782  ..., -52.42448381 -54.90990381
  -54.88467196]
 [-58.43349891 -53.15541452 -51.71927187 ..., -53.17606964 -57.90114494
  -56.63809808]
 [-63.51685582 -66.48714233 -67.9205193  ..., -62.23625721 -68.1013725
  -60.99568525]]


In [67]:
y_harmonic, y_percussive = librosa.effects.hpss(y)
print (y_harmonic)
print (y_percussive)
# Decompose an audio time series into harmonic and percussive components.
# Ref: https://librosa.github.io/librosa/generated/librosa.effects.hpss.html

[-0.00296632 -0.00296497 -0.00238975 ...,  0.          0.          0.        ]
[ 0.00164433  0.0014343   0.00183993 ...,  0.          0.          0.        ]


In [66]:
# What do the spectrograms look like?
# Let's make and display a mel-scaled power (energy-squared) spectrogram
S_harmonic   = librosa.feature.melspectrogram(y_harmonic, sr=sr)
S_percussive = librosa.feature.melspectrogram(y_percussive, sr=sr)

# Convert to log scale (dB). We'll use the peak power as reference.
log_Sh = librosa.power_to_db(S_harmonic, ref=np.max)
log_Sp = librosa.power_to_db(S_percussive, ref=np.max)
print (log_Sh)
print (log_Sp)

[[-24.99103348 -25.97891754 -30.73029777 ..., -26.78943487 -27.8451456
  -30.9201283 ]
 [-23.82896081 -21.33987971 -22.88799333 ..., -22.67792561 -19.4839626
  -20.30267438]
 [-23.33195266 -18.33794502 -12.1974036  ..., -14.18689003 -13.16274011
  -13.71835991]
 ..., 
 [-58.60952141 -56.0036429  -54.33478349 ..., -58.03683865 -57.13397223
  -56.9902433 ]
 [-62.27440894 -58.79613293 -57.74104194 ..., -60.22538074 -61.2638563
  -63.09457427]
 [-70.30121275 -71.62333805 -71.73139351 ..., -71.37062239 -72.69178378
  -68.25014399]]
[[-19.88054857 -19.36600323 -22.60524266 ..., -23.60885351 -27.56514447
  -30.3227642 ]
 [-23.71630623 -19.3292895  -21.23735677 ..., -24.31210692 -23.76702877
  -27.98649423]
 [-25.59812187 -21.17191218 -17.20595055 ..., -25.23929538 -26.98885428
  -28.78194942]
 ..., 
 [-58.25137682 -51.25464326 -47.15979212 ..., -53.75575464 -60.96397974
  -64.78100097]
 [-61.88725201 -54.84535923 -53.05819981 ..., -53.61134344 -62.48248138
  -67.85573332]
 [-63.37822249 -67.8

In [24]:
# We'll use a CQT-based chromagram here.  An STFT-based implementation also exists in chroma_cqt()
# We'll use the harmonic component to avoid pollution from transients
C = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr)

In [32]:
# Next, we'll extract the top 13 Mel-frequency cepstral coefficients (MFCCs)
mfcc        = librosa.feature.mfcc(S=log_S, n_mfcc=13)

# Let's pad on the first and second deltas while we're at it
delta_mfcc  = librosa.feature.delta(mfcc)
delta2_mfcc = librosa.feature.delta(mfcc, order=2)
# For future use, we'll stack these together into one matrix
M = np.vstack([mfcc, delta_mfcc, delta2_mfcc])

In [33]:
# Now, let's run the beat tracker.
# We'll use the percussive component for this part
plt.figure(figsize=(12, 6))
tempo, beats = librosa.beat.beat_track(y=y_percussive, sr=sr)

In [34]:
print('Estimated tempo:        %.2f BPM' % tempo)

print('First 5 beat frames:   ', beats[:5])

# Frame numbers are great and all, but when do those beats occur?
print('First 5 beat times:    ', librosa.frames_to_time(beats[:5], sr=sr))

# We could also get frame numbers from times by librosa.time_to_frames()

Estimated tempo:        136.00 BPM
First 5 beat frames:    [17 36 55 75 95]
First 5 beat times:     [ 0.39473923  0.83591837  1.27709751  1.7414966   2.20589569]


In [37]:
# feature.sync will summarize each beat event by the mean feature vector within that beat

M_sync = librosa.util.sync(M, beats)

In [40]:
# Beat synchronization is flexible.
# Instead of computing the mean delta-MFCC within each beat, let's do beat-synchronous chroma
# We can replace the mean with any statistical aggregation function, such as min, max, or median.

C_sync = librosa.util.sync(C, beats, aggregate=np.median)

In [58]:
input_file = "C:/Users/Ujjval/Downloads/ESC-50-master/ESC-50-master/audio/1-7057-A-12.wav" 
output_file = "C:/Users/Ujjval/Desktop/Desktop/SJSU/Courses/255/Project/new.csv"

In [59]:
#!/usr/bin/env python
'''
CREATED:2013-02-11 18:37:30 by Brian McFee <brm2132@columbia.edu>
Track beat events in an audio file
Usage:   ./beat_tracker.py [-h] input_file.mp3    output_beats.csv
'''
from __future__ import print_function

import argparse
import sys
import librosa

print('Loading ', input_file)
y, sr = librosa.load(input_file, sr=22050)

# Use a default hop size of 512 samples @ 22KHz ~= 23ms
hop_length = 512

# This is the window length used by default in stft
print('Tracking beats')
tempo, beats = librosa.beat.beat_track(y=y, sr=sr, hop_length=hop_length)

print('Estimated tempo: {:0.2f} beats per minute'.format(tempo))

# save output
# 'beats' will contain the frame numbers of beat events.
beat_times = librosa.frames_to_time(beats, sr=sr, hop_length=hop_length)

print('done!')
beat_times

Loading  C:/Users/Ujjval/Downloads/ESC-50-master/ESC-50-master/audio/1-7057-A-12.wav
Tracking beats
Estimated tempo: 184.57 beats per minute
done!


array([ 0.06965986,  0.39473923,  0.67337868,  0.9752381 ,  1.27709751,
        1.57895692,  1.88081633,  2.20589569,  2.5077551 ,  2.80961451,
        3.11147392,  3.41333333,  3.7384127 ,  4.06349206,  4.36535147,
        4.66721088])